# Avoiding Loops: Advanced Patterns

**Module 04 | Notebook 03**

---

## Objective
By the end of this notebook, you will master:
- Advanced loop elimination patterns
- np.einsum for complex operations
- np.apply_along_axis and friends
- Ufunc methods (reduce, accumulate, outer)
- Real-world case studies

In [2]:
import numpy as np
import time
np.set_printoptions(precision=3)

---
## 1. numpy.einsum - Einstein Summation

In [3]:
# einsum: express complex multi-array operations concisely
# Syntax: 'input_subscripts->output_subscripts'

# Example: Matrix multiplication
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

# C[i,k] = sum_j A[i,j] * B[j,k]
C = np.einsum('ij,jk->ik', A, B)
print(f"A @ B using einsum:\n{C}")
print(f"Verify with @:\n{A @ B}")

A @ B using einsum:
[[19 22]
 [43 50]]
Verify with @:
[[19 22]
 [43 50]]


In [4]:
# Common einsum patterns
A = np.arange(12).reshape(3, 4)
v = np.arange(4)

# Sum all elements
print(f"Sum all: {np.einsum('ij->', A)} = {A.sum()}")

# Sum along axis 0 (columns)
print(f"Sum cols: {np.einsum('ij->j', A)} = {A.sum(axis=0)}")

# Sum along axis 1 (rows)
print(f"Sum rows: {np.einsum('ij->i', A)} = {A.sum(axis=1)}")

# Transpose
print(f"Transpose: \n{np.einsum('ij->ji', A)}")

# Trace (sum of diagonal)
B = np.arange(9).reshape(3, 3)
print(f"Trace: {np.einsum('ii->', B)} = {np.trace(B)}")

Sum all: 66 = 66
Sum cols: [12 15 18 21] = [12 15 18 21]
Sum rows: [ 6 22 38] = [ 6 22 38]
Transpose: 
[[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]
Trace: 12 = 12


In [5]:
# Dot product
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

print(f"Dot product: {np.einsum('i,i->', a, b)} = {np.dot(a, b)}")

Dot product: 32 = 32


In [6]:
# Outer product
print(f"Outer product:\n{np.einsum('i,j->ij', a, b)}")
print(f"Verify:\n{np.outer(a, b)}")

Outer product:
[[ 4  5  6]
 [ 8 10 12]
 [12 15 18]]
Verify:
[[ 4  5  6]
 [ 8 10 12]
 [12 15 18]]


In [7]:
# Batch matrix multiplication
# Shape: (batch, m, n) @ (batch, n, k) -> (batch, m, k)
batch = 5
A = np.random.rand(batch, 3, 4)
B = np.random.rand(batch, 4, 2)

# einsum handles batch dimension automatically
C = np.einsum('bij,bjk->bik', A, B)
print(f"A shape: {A.shape}")
print(f"B shape: {B.shape}")
print(f"C shape: {C.shape}")

A shape: (5, 3, 4)
B shape: (5, 4, 2)
C shape: (5, 3, 2)


In [8]:
# Element-wise multiplication then sum (common in attention)
A = np.random.rand(10, 5)
B = np.random.rand(10, 5)

# Row-wise dot products
dots = np.einsum('ij,ij->i', A, B)  # Shape (10,)
print(f"Row-wise dots shape: {dots.shape}")

# Verify
dots_manual = (A * B).sum(axis=1)
print(f"Match: {np.allclose(dots, dots_manual)}")

Row-wise dots shape: (10,)
Match: True


---
## 2. Ufunc Methods

In [9]:
# Universal functions have special methods
arr = np.array([1, 2, 3, 4, 5])

# reduce: apply along axis, collapsing to scalar
print(f"add.reduce (sum): {np.add.reduce(arr)}")
print(f"multiply.reduce (prod): {np.multiply.reduce(arr)}")
print(f"maximum.reduce (max): {np.maximum.reduce(arr)}")

add.reduce (sum): 15
multiply.reduce (prod): 120
maximum.reduce (max): 5


In [10]:
# accumulate: cumulative version
print(f"add.accumulate (cumsum): {np.add.accumulate(arr)}")
print(f"multiply.accumulate (cumprod): {np.multiply.accumulate(arr)}")
print(f"maximum.accumulate (cummax): {np.maximum.accumulate(arr)}")

add.accumulate (cumsum): [ 1  3  6 10 15]
multiply.accumulate (cumprod): [  1   2   6  24 120]
maximum.accumulate (cummax): [1 2 3 4 5]


In [11]:
# outer: cartesian product operation
a = np.array([1, 2, 3])
b = np.array([10, 20])

print(f"add.outer:\n{np.add.outer(a, b)}")
print(f"multiply.outer:\n{np.multiply.outer(a, b)}")

add.outer:
[[11 21]
 [12 22]
 [13 23]]
multiply.outer:
[[10 20]
 [20 40]
 [30 60]]


In [12]:
# reduceat: reduce at specific indices (grouped reduction)
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8])
indices = [0, 2, 5]  # Start positions of groups

# Sum elements in groups: [0:2], [2:5], [5:8]
sums = np.add.reduceat(arr, indices)
print(f"Array: {arr}")
print(f"Group sums at {indices}: {sums}")
# [1+2=3, 3+4+5=12, 6+7+8=21]

Array: [1 2 3 4 5 6 7 8]
Group sums at [0, 2, 5]: [ 3 12 21]


In [13]:
# at: in-place operation at indices (with accumulation)
arr = np.zeros(5)
indices = np.array([0, 2, 2, 1])
values = np.array([1, 1, 1, 1])

np.add.at(arr, indices, values)
print(f"Result after add.at: {arr}")
# Index 2 appears twice, so gets 2

Result after add.at: [1. 1. 2. 0. 0.]


---
## 3. Apply Functions Along Axis

In [14]:
# np.apply_along_axis: apply function to 1D slices
# Use when you need custom function but want to avoid loops

def my_func(x):
    """Custom function: range of values."""
    return x.max() - x.min()

arr = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Apply along axis 0 (each column)
result_cols = np.apply_along_axis(my_func, 0, arr)
print(f"Range per column: {result_cols}")

# Apply along axis 1 (each row)
result_rows = np.apply_along_axis(my_func, 1, arr)
print(f"Range per row: {result_rows}")

Range per column: [6 6 6]
Range per row: [2 2 2]


In [15]:
# Note: apply_along_axis is NOT truly vectorized
# Use built-in functions when possible

# Better: use np.ptp (peak-to-peak)
print(f"Using np.ptp: {np.ptp(arr, axis=0)}")

Using np.ptp: [6 6 6]


In [16]:
# np.apply_over_axes: reduce over multiple axes
arr = np.arange(24).reshape(2, 3, 4)
print(f"Original shape: {arr.shape}")

# Sum over axes 0 and 2
result = np.apply_over_axes(np.sum, arr, [0, 2])
print(f"Sum over axes [0,2] shape: {result.shape}")
print(f"Result: {result.flatten()}")

Original shape: (2, 3, 4)
Sum over axes [0,2] shape: (1, 3, 1)
Result: [ 60  92 124]


---
## 4. Advanced Loop-Free Patterns

In [17]:
# Pattern: Rolling/Sliding window WITHOUT loop
from numpy.lib.stride_tricks import as_strided

arr = np.arange(10)
window_size = 3
n_windows = len(arr) - window_size + 1

# Create view with sliding windows
shape = (n_windows, window_size)
strides = (arr.strides[0], arr.strides[0])

windows = as_strided(arr, shape=shape, strides=strides)
print(f"Windows:\n{windows}")
print(f"Moving sum: {windows.sum(axis=1)}")

Windows:
[[0 1 2]
 [1 2 3]
 [2 3 4]
 [3 4 5]
 [4 5 6]
 [5 6 7]
 [6 7 8]
 [7 8 9]]
Moving sum: [ 3  6  9 12 15 18 21 24]


In [18]:
# Pattern: Pairwise distances (no nested loops)
points = np.array([[0, 0], [1, 0], [0, 1], [1, 1]])
n = len(points)

# Use broadcasting: (n,1,2) - (1,n,2) -> (n,n,2)
diff = points[:, np.newaxis, :] - points[np.newaxis, :, :]
distances = np.linalg.norm(diff, axis=2)
print(f"Distance matrix:\n{distances}")

Distance matrix:
[[0.    1.    1.    1.414]
 [1.    0.    1.414 1.   ]
 [1.    1.414 0.    1.   ]
 [1.414 1.    1.    0.   ]]


In [19]:
# Pattern: Running minimum/maximum
arr = np.array([3, 1, 4, 1, 5, 9, 2, 6])

# Running minimum
running_min = np.minimum.accumulate(arr)
print(f"Array: {arr}")
print(f"Running min: {running_min}")

# Running maximum
running_max = np.maximum.accumulate(arr)
print(f"Running max: {running_max}")

Array: [3 1 4 1 5 9 2 6]
Running min: [3 1 1 1 1 1 1 1]
Running max: [3 3 4 4 5 9 9 9]


In [20]:
# Pattern: Leaky integration / exponential smoothing
# y[i] = alpha * x[i] + (1-alpha) * y[i-1]
# This needs a loop, but we can use ufunc approach

def exponential_smoothing(x, alpha):
    """Vectorized version using cumsum trick."""
    n = len(x)
    powers = (1 - alpha) ** np.arange(n)
    
    # Apply weights
    result = np.zeros(n)
    for i in range(n):
        weights = powers[:i+1][::-1]
        result[i] = alpha * np.sum(x[:i+1] * weights)
    return result

# For truly vectorized, use scipy.lfilter or numba
x = np.array([1, 2, 3, 4, 5], dtype=float)
print(f"Input: {x}")

Input: [1. 2. 3. 4. 5.]


In [21]:
# Pattern: Index operations without loops
# Find indices where condition changes
arr = np.array([0, 0, 1, 1, 1, 0, 0, 1, 0])

# Find where values change
change_points = np.where(np.diff(arr) != 0)[0] + 1
print(f"Array: {arr}")
print(f"Change points: {change_points}")

Array: [0 0 1 1 1 0 0 1 0]
Change points: [2 5 7 8]


In [22]:
# Pattern: Segment-wise operations using reduceat
values = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
# Groups: [0:3], [3:7], [7:10]
starts = np.array([0, 3, 7])

# Sum each segment
segment_sums = np.add.reduceat(values, starts)

# Mean each segment
segment_lens = np.diff(np.append(starts, len(values)))
segment_means = segment_sums / segment_lens

print(f"Values: {values}")
print(f"Segment sums: {segment_sums}")
print(f"Segment means: {segment_means}")

Values: [ 1  2  3  4  5  6  7  8  9 10]
Segment sums: [ 6 22 27]
Segment means: [2.  5.5 9. ]


---
## 5. Real-World Case Studies

In [23]:
# Case 1: Cosine similarity matrix (NLP embeddings)
embeddings = np.random.rand(100, 768)  # 100 sentences, 768-dim

# Normalize
norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
normalized = embeddings / norms

# Full similarity matrix
similarity_matrix = normalized @ normalized.T
print(f"Similarity matrix shape: {similarity_matrix.shape}")
print(f"Sample similarities: {similarity_matrix[0, :5]}")

Similarity matrix shape: (100, 100)
Sample similarities: [1.    0.745 0.743 0.746 0.764]


In [24]:
# Case 2: Attention mechanism (simplified)
seq_len = 10
d_model = 64

Q = np.random.randn(seq_len, d_model)  # Query
K = np.random.randn(seq_len, d_model)  # Key
V = np.random.randn(seq_len, d_model)  # Value

# Attention scores
scores = Q @ K.T / np.sqrt(d_model)

# Softmax
exp_scores = np.exp(scores - scores.max(axis=-1, keepdims=True))
attention = exp_scores / exp_scores.sum(axis=-1, keepdims=True)

# Output
output = attention @ V
print(f"Output shape: {output.shape}")

Output shape: (10, 64)


In [25]:
# Case 3: K-nearest neighbors (vectorized)
n_samples = 100
n_features = 5
k = 3

data = np.random.rand(n_samples, n_features)
query = np.random.rand(n_features)

# All distances at once
distances = np.linalg.norm(data - query, axis=1)

# K nearest indices
k_nearest_idx = np.argpartition(distances, k)[:k]
print(f"K nearest indices: {k_nearest_idx}")
print(f"K nearest distances: {distances[k_nearest_idx]}")

K nearest indices: [ 6 22 42]
K nearest distances: [0.365 0.456 0.461]


In [26]:
# Case 4: Image convolution (simplified 2D)
from numpy.lib.stride_tricks import sliding_window_view

image = np.random.rand(28, 28)  # Grayscale image
kernel = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])  # Sobel

# Create patches
patches = sliding_window_view(image, kernel.shape)
print(f"Patches shape: {patches.shape}")

# Convolution via einsum
result = np.einsum('ijkl,kl->ij', patches, kernel)
print(f"Convolution result shape: {result.shape}")

Patches shape: (26, 26, 3, 3)
Convolution result shape: (26, 26)


---
## Key Points Summary

**np.einsum:**
- Express complex multi-array operations concisely
- `'ij,jk->ik'` for matrix multiply
- Handles batched operations naturally

**Ufunc Methods:**
- `.reduce()`: Collapse along axis
- `.accumulate()`: Cumulative version
- `.outer()`: Cartesian product
- `.reduceat()`: Grouped reduction
- `.at()`: In-place accumulation

**When Loops Are OK:**
- Complex state dependencies
- Small arrays (overhead dominates)
- Readability trumps speed
- Use numba/cython for critical loops

---
## Interview Tips

**Q1: What is np.einsum and when would you use it?**
> einsum (Einstein summation) expresses multi-array operations using index notation. Use for batch matrix ops, element-wise multiply+sum, and complex tensor operations. It can replace multiple NumPy calls with a single operation.

**Q2: How do ufunc.reduce and ufunc.accumulate differ?**
> - `reduce` collapses an axis to produce a single value (like sum)
> - `accumulate` keeps intermediate results (like cumsum)

**Q3: Name some operations that cannot be vectorized?**
> - Recurrent operations with dependencies (y[i] depends on y[i-1])
> - Conditional early termination
> - Complex branching per element

**Q4: What is reduceat used for?**
> Grouped reductions without loop. Given indices [0,3,7], it computes the reduction for segments [0:3], [3:7], [7:end]. Useful for segment-wise statistics.

---
## Practice Exercises

### Exercise 1: Batch matrix transpose using einsum

In [27]:
# Transpose each matrix in batch
batch = np.random.rand(5, 3, 4)  # 5 matrices of shape 3x4
# Expected output shape: (5, 4, 3)


In [28]:
# Solution
batch = np.random.rand(5, 3, 4)
transposed = np.einsum('bij->bji', batch)
print(f"Original: {batch.shape}")
print(f"Transposed: {transposed.shape}")

Original: (5, 3, 4)
Transposed: (5, 4, 3)


### Exercise 2: Cumulative maximum difference

In [29]:
# Find max profit in stock prices: max(price[j] - price[i]) where j > i
# This is: max(price - running_min)
prices = np.array([7, 1, 5, 3, 6, 4])


In [30]:
# Solution
prices = np.array([7, 1, 5, 3, 6, 4])

running_min = np.minimum.accumulate(prices)
profit = prices - running_min
max_profit = profit.max()

print(f"Prices: {prices}")
print(f"Running min: {running_min}")
print(f"Profit at each point: {profit}")
print(f"Max profit: {max_profit}")

Prices: [7 1 5 3 6 4]
Running min: [7 1 1 1 1 1]
Profit at each point: [0 0 4 2 5 3]
Max profit: 5


### Exercise 3: Group by sum using reduceat

In [31]:
# Values with group IDs, compute sum per group
values = np.array([10, 20, 30, 40, 50, 60])
groups = np.array([0, 0, 1, 1, 1, 2])  # 3 groups


In [32]:
# Solution
values = np.array([10, 20, 30, 40, 50, 60])
groups = np.array([0, 0, 1, 1, 1, 2])

# Get start indices of each group
unique_groups, indices = np.unique(groups, return_index=True)
group_sums = np.add.reduceat(values, indices)

print(f"Groups: {unique_groups}")
print(f"Sums: {group_sums}")

# Alternative using bincount
group_sums_alt = np.bincount(groups, weights=values)
print(f"Using bincount: {group_sums_alt}")

Groups: [0 1 2]
Sums: [ 30 120  60]
Using bincount: [ 30. 120.  60.]


---
## Module 04 Complete!

You have mastered Broadcasting and Vectorization:
- Broadcasting Fundamentals
- Vectorization Techniques
- Avoiding Loops

**Next Module:** 05_advanced_indexing - Fancy indexing, boolean indexing, and advanced selection!